In [2]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')
import numpy as np

import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/Huazhu/'):
    print(f)
    date=f.split('_')[-1].split('.')[0]
    d=pd.read_csv('./DataSource/Huazhu\%s' % f)


    c1=d['brandLevel'].isin(['经济型','舒适型'])

    d1=d[c1][['cityId', 'cityName', 'brandLevel', 'hotelId', 'hotelName', 'roomName','checkInType', 'stock']]
    d1=d1.groupby(by=['cityName', 'brandLevel','hotelName','checkInType','roomName']).mean().reset_index()
    d1=d1.groupby(by=['cityName', 'brandLevel','checkInType','hotelName']).sum().reset_index()
    # pv=pd.pivot_table(d1,index='cityName',values='stock',columns='checkInType', aggfunc=np.sum).reset_index()
    pv=pd.pivot_table(d1,index=['cityName','hotelName'],values='stock',columns='checkInType', aggfunc=np.sum).reset_index()
    pv=pv[~pv['now'].isna()]
    pv=pv[~pv['future'].isna()]
    
    pv['bookedNights']=pv['future']-pv['now']
    pv['hotel_occ']=pv['bookedNights']/pv['future']
    
    c2=pv['now']>5
    c3=pv['hotel_occ']<0.9

    pv=pv[c2&c3]

    pv=pv.groupby(by='cityName').sum().reset_index()
    pv['city_occ']=pv['bookedNights']/pv['future']
    pv['date']=date

    dfo=pd.concat([dfo,pv],axis=0)

old=pd.read_hdf(r'Processed_Data/hotel.h5',key='1223')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/hotel.h5',key='1223')

huazhu_stock_2023-01-02.csv
huazhu_stock_2023-01-03.csv


In [11]:
#prepare DF
hotel=dfo.copy()
hotel['date']=pd.to_datetime(hotel['date'])
hotel['cityName']=hotel['cityName'].str.replace('市','')

hotel1=hotel.sort_values(by=['cityName','date'])
hotel1['bookedNights_last7d_mean']=hotel1.groupby('cityName')['bookedNights'].transform(lambda x: x.rolling(7, 7).mean())
hotel1['bookedNights_change_WoW']=(hotel1['bookedNights_last7d_mean']/hotel1.groupby('cityName')['bookedNights_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1

c1=hotel1['date'].astype('str')< '2022-07-01'
c2=hotel1['bookedNights']>0
june_mean=hotel1[c1&c2].groupby('cityName').mean().reset_index().sort_values(by='bookedNights')[['cityName','bookedNights']].reset_index(drop=True)
june_mean.columns=['cityName', 'bookedNights_june']

hotel1=hotel1.merge(june_mean,on='cityName')
hotel1['changes_since_june']=hotel1['bookedNights_last7d_mean']/hotel1['bookedNights_june']

In [9]:
# covid rank vs. hotel nights WoW
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
# riqi='2022-12-29'

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')

hotel2=hotel1.merge(df_covid,left_on=['cityName'],right_on=['城市'],how='left')

hotel2['date']=hotel2['date'].astype('string')
hotel2=hotel2[hotel2['date']==riqi].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel2['bookedNights_change_WoW']<2].head(70).reset_index()

o=hotel2[['cityName',  'progress_2','bookedNights_change_WoW']]


o['size']=np.nan
o['group']='其他'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-01-03


In [12]:
# covid rank vs. hotel nights vs 202206

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')

hotel2=hotel1.merge(df_covid,left_on=['cityName'],right_on=['城市'],how='left')

hotel2['date']=hotel2['date'].astype('string')
hotel2=hotel2[hotel2['date']==riqi].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel2['changes_since_june']<2].head(70).reset_index()

o=hotel2[['cityName',  'progress_2','changes_since_june']]


o['size']=np.nan
o['group']='其他'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

d=hotel1[hotel1['date']==riqi][['cityName','date','changes_since_june']].reset_index(drop=True).rename(columns={'cityName':'城市','changes_since_june':'hotel'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='hotel')
# d

2023-01-03


In [13]:
# GDP vs. hotel nights WoW

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

hotel2=hotel1.merge(city_popu,left_on=['cityName'],right_on=['城市'],how='left')

hotel2['date']=hotel2['date'].astype('string')
hotel2=hotel2[hotel2['date']==riqi].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel2['bookedNights_change_WoW']<2].head(70).reset_index()

o=hotel2[['cityName',  'GDP','bookedNights_change_WoW']]


o['size']=np.nan
o['group']='其他'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-03


In [14]:
# GDP vs. hotel nights vs 202206

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

hotel2=hotel1.merge(city_popu,left_on=['cityName'],right_on=['城市'],how='left')

hotel2['date']=hotel2['date'].astype('string')
hotel2=hotel2[hotel2['date']==riqi].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel2['changes_since_june']<2].head(70).reset_index()

o=hotel2[['cityName',  'GDP','changes_since_june']]


o['size']=np.nan
o['group']='其他'

c1=o['cityName'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-03


In [13]:
d1=pd.read_clipboard()
d2=pd.pivot_table(new[new['date']>'2022-12-28'],index='cityName',columns='date',values='bookedNights',aggfunc='sum').reset_index()
d1.merge(d2,left_on='城市',right_on='cityName',how='left').to_clipboard(index=False)